In [67]:
# Imports... Cool kids import on one line
from lxml import html
import requests, re, folium, webbrowser
from bs4 import BeautifulSoup

# The Stuff Object
This program assumes that stuff ought to be free. Otherwise, why would I want it?
## in Version 2 images work ;)

In [64]:
class Stuff(object):
    thing = ""
    url = ""
    location = ""
    image = ""
    #constructor the de-structor!!  
    def __init__(self, thing, url, location, image):
        self.thing = thing
        self.url = 'http://montreal.craigslist.ca' + url
        self.location = location
        self.image = image #this isn't implemented yet

    #the stringifing printer.... Python is so pretty
    def __str__(self):
        return " what: %s \n where: %s \n link: %s \n image: %s" % (self.thing, self.location, self.url, self.image)

## Scrape Functions
* So the first step is to find the user's city.
* The second step is to gather the different lists of desirable/useful information (ie title, url, location, and image url).

### Location of User

In [56]:
def setup_place():
    user_place = input("What major city are you near? (or, 'help') ")
    if user_place == "help":
        print("craigslist serves many major cities, and the peripheral neighborhoods, try something like 'montreal' or 'newyork'")
        user_place = input("What major city are you near? ")
    return user_place 

### Soup

In [57]:
def setup_page(user_place):
    free_url = 'http://' + user_place +'.craigslist.com/search/zip'
    try:
        free_page = requests.get(free_url)
        soup = BeautifulSoup(free_page.text)
    except:
        print("something when wrong")
    return soup

### Location of Stuff and Url

In [58]:
def get_locations(user_place, soup):
    free_locations = []
    for span in soup.find_all("span", class_="pnr"):
        loc_node = str(span.find('small'))
        if loc_node == "None":
            _loc = user_place +", Somewhere"
        else:
            _loc = loc_node.strip('<small ()</small>')
            _loc = _loc + ", " + user_place
        #print(_loc)#
        free_locations.append(_loc)
    return free_locations

def get_urls(soup):
    free_urls = []
    for row in soup.find_all("a", class_="i"):
        _url = row['href']
        free_urls.append(_url)
    return free_urls

### Images and Titles

In [59]:
def get_images(soup):
    free_images = []
    for row in soup.find_all("a", class_="i"):
        try:
            _img = str(row['data-ids'])
            _img = _img[2:]
            _img = "https://images.craigslist.org/" + _img + "_300x300.jpg"
        except:
            _img = "no image"
        free_images.append(_img)
    return free_images

def get_things(soup): # Thing is images
    free_things = []
    for node in soup.find_all("a", class_="hdrlnk"):
        _thing = node.get_text()
        free_things.append(_thing)
    return free_things

**SO** what follows is to pass these *lists* into the constructor Stuff(blah), and wind up with a list of stuffs...

## Constructor the De-Structor List Combobulator

In [65]:
"""Set Up"""
place = setup_place() # user place
soup = setup_page(place) # soup, needs the user place for request
"""Construction"""
locs = get_locations(place, soup) # locations, needs user place for fine tuning
urls = get_urls(soup) # urls of stuff
things = get_things(soup) # things of stuff
images = get_images(soup) # get_images(soup) # WILL THIS EVERY WORK?

"""Constructor Combobulator"""
freestuffs = [Stuff(things[x], urls[x], locs[x], images[x]) for x in range(0,20)] 

What major city are you near? (or, 'help') montreal


In [66]:
print(freestuffs[3])

 what: FREE Magazines by the pallet 
 where: montreal, Somewhere 
 link: http://montreal.craigslist.ca/zip/5041940898.html 
 image: https://images.craigslist.org/00A0A_cWTj0ARWrS4_300x300.jpg


## And then?
Take this list of freestuffs and pass it into a map cruncher (ie. mappify) with the pretty pictures and all